# **Finding similar items**

### Project for the **Algorithms for massive data course**


MSc, Data Science for Economics*

Shojaat Joodi Bigdilo

June 2024

In [5]:
# connecting my Google Drive and google colab
from google.colab import drive
drive.mount('/content/gdrive',force_remount=True)

Mounted at /content/gdrive


In [6]:
!pip install kaggle

import warnings
warnings.filterwarnings("ignore")

In [ ]:
!pip install findspark

In [ ]:
!pip install pyspark

In [9]:
# connecting to Kaggle
import os
os.environ['KAGGLE_USERNAME'] = 'xxxxxxxxx'

os.environ['KAGGLE_KEY'] = 'xxxxxxxxx' 

In [10]:
!kaggle datasets download -d asaniczka/1-3m-linkedin-jobs-and-skills-2024

In [11]:
extract_to_path  = "/content/gdrive/My Drive/Massive_Data_Project/Job_Dataset"

import zipfile
with zipfile.ZipFile('1-3m-linkedin-jobs-and-skills-2024.zip', 'r') as zip_ref:
    zip_ref.extractall(extract_to_path)

In [12]:
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.functions import isnan, when, count, col, countDistinct, udf
from pyspark.ml.feature import Tokenizer, StopWordsRemover, HashingTF, IDF, MinHashLSH, Normalizer
from pyspark.sql.types import DoubleType

from pyspark.ml.linalg import Vectors, VectorUDT
from pyspark.sql.functions import explode
from pyspark.sql.types import StringType, ArrayType, StructType, StructField, IntegerType

from pyspark.sql.functions import lower, regexp_replace, size
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType, IntegerType

import re
import time
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure

import string

import datetime
import warnings
warnings.filterwarnings("ignore")




In [ ]:
# SparkSession initialization
conf = SparkConf().setAppName("Similar_Documents")
spark = SparkSession.builder.enableHiveSupport().config(conf = conf).getOrCreate()
sc = spark.sparkContext
type(sc)

In [14]:
# Reading Dataset from Google Drive
file_path = "/content/gdrive/My Drive/Massive_Data_Project/Job_Dataset/job_summary.csv"

df_Dataset = spark.read.csv(file_path, header=True, inferSchema=True, multiLine=True, escape='"',
                           encoding = "ISO-8859-1")

## choosing chunk of data

In [15]:
# Creating Chunk of Dataset
import pandas as pd

size = 5000
df_Dataset_2 = df_Dataset.limit(size)

# Convert Spark DataFrame to Pandas DataFrame
df_Dataset_2 = df_Dataset_2.toPandas()
df_Dataset_2.to_csv('/content/gdrive/My Drive/Massive_Data_Project/Job_Dataset/Chunk5000.csv', index=False)

In [16]:
# Reading Chunk Dataset
file_path = "/content/gdrive/My Drive/Massive_Data_Project/Job_Dataset/Chunk5000.csv"

Job_Dataset = spark.read.csv(file_path, header=True, inferSchema=True, multiLine=True, escape='"',
                           encoding = "ISO-8859-1")


In [ ]:
type(Job_Dataset)

# Pre-processsing

### Exploratory analysis

In [ ]:
Job_Dataset.show(n = 10)

In [ ]:
Job_Dataset = Job_Dataset.select("job_summary")
Job_Dataset.show(n = 10)

#### Giving Id for each row

In [20]:
indexed_rdd = Job_Dataset.rdd.zipWithIndex()
Job_Dataset = indexed_rdd.map(lambda x: (x[1], x[0][0])).toDF(["Id", "job_summary"])

In [ ]:
Job_Dataset.show(5)

In [22]:
Job_df = Job_Dataset

In [ ]:
Job_df.show(n = 5)

In [ ]:
# checking missing values in the columns
Job_df.select([count(when(isnan(c), c)).alias(c) for c in Job_df.columns]).show()

In [ ]:
#count distinct values in each column
Job_df.select([countDistinct(c).alias(c) for c in Job_df.columns]).show()

### Duplicates check

In [ ]:
# show duplicates in Body column
Job_df.groupBy("job_summary").count().filter("count > 1").show()

In [ ]:
# Filter the rows where 'job_summary' starts with 'Job Title:\nCerti'
filtered_rows = Job_df.filter(col("job_summary").startswith("Job Title:\nCertified Nursing Assistant (CNA)\nCompany"))
filtered_rows.show()

In [ ]:
# ID number 1319
row_with_id_1319 = Job_df.filter(Job_df['ID'] == 1319).collect()

txt = row_with_id_1319[0][1:][0]
txt

In [ ]:
# ID number 1586
row_with_id_1586 = Job_df.filter(Job_df['ID'] == 1586).collect()

txt2 = row_with_id_1586[0][1:][0]
txt2

In [ ]:
# Checking Equality of texts
if txt == txt2:
    print('Equal')
else:
    print('Not Equal')

### Delete Duplicates Document

In [31]:
# Optionally, delete duplicates based on a specific column
Job_df = Job_df.dropDuplicates(['job_summary'])

In [ ]:
#count distinct values in each column
Job_df.select([countDistinct(c).alias(c) for c in Job_df.columns]).show()

In [ ]:
# checking again duplicates
Job_df.groupBy("job_summary").count().filter("count > 1").show()

In [ ]:
row_with_id_1586 = Job_df.filter(Job_df['ID'] == 1586).collect()
row_with_id_1586

# Text cleaning and pre-processing

In [35]:
Job_df = Job_df.select('Id',"job_summary")
# questions_body.show(n = 10)

### LoweCasing Text

In [36]:
Job_df = Job_df.withColumn('job_summary', lower(Job_df['job_summary']))

### Remove HTML Tags

In [37]:
def remove_html_tags(text):
    pattern = re.compile('<.*?>')
    return pattern.sub(r'', text) if text else text

# Register the function as a UDF
remove_html_tags_udf = udf(remove_html_tags, StringType())

# Apply the UDF to the job_summary column
Job_df = Job_df.withColumn('job_summary', remove_html_tags_udf(Job_df['job_summary']))

In [38]:
# Job_df.show(n = 5 , truncate=False)

###  Remove URLs

In [39]:
# Here We also Use Regular Expressions to Remove URLs from Text or Whole Corpus.
def remove_url(text):
    pattern = re.compile(r'https?://\S+|www\.\S+')
    return pattern.sub(r'', text)

remove_url_udf = udf(remove_url, StringType())
Job_df = Job_df.withColumn('job_summary', remove_url_udf(Job_df['job_summary']))

In [40]:
# Job_df.show(n = 5 , truncate=False)

### Remove Punctuations

In [41]:
import string
def remove_punctuation(text):
    return text.translate(str.maketrans('', '', string.punctuation))


remove_punctuation_udf = udf(remove_punctuation, StringType())
Job_df = Job_df.withColumn('job_summary', remove_punctuation_udf(Job_df['job_summary']))

In [42]:
# Job_df.show(n = 5 , truncate=False)

### Remove numbers

The following document has aroud 42 different number inside it, so we need to delet them.
3x12 , 180000060000, 12003, 0, 4, 02142024, 05152024, 13, 556166975, 56166975 , 12 , 7 , 7, 100 , 133, 3467, 68100, 10 , 25, 50 , 100,
100 , 20 , 3, 2, , 1, 0, 100, 15, 15, 15, 91, 401,36, 50, 2023, 2022, 2021 ,2020, 2019.

In [43]:
# row_with_id_160 = Job_df.filter(Job_df['ID'] == 160).collect()
# row_with_id_160

In [44]:
def remove_numbers(text):
    pattern = re.compile(r'\d+')
    return pattern.sub(r'', text)

remove_numbers_udf = udf(remove_numbers, StringType())
Job_df = Job_df.withColumn('job_summary', remove_numbers_udf(Job_df['job_summary']))

In [45]:
# row_with_id_160 = Job_df.filter(Job_df['ID'] == 160).collect()
# row_with_id_160

### Remove Non-ASCII characters:
Some texts have some non-ASCII characters like (ã°â\x9fâ\x9fâ¡), so we need to delete them from texts

In [46]:
row_with_id_915 = Job_df.filter(Job_df['ID'] == 915).collect()
row_with_id_915

In [47]:
def remove_non_ascii(text):
    if text is None:
        return None
    return re.sub(r'[^\x00-\x7F]+', '', str(text))

remove_non_ascii_udf = udf(remove_non_ascii, StringType())
Job_df = Job_df.withColumn('job_summary', remove_non_ascii_udf(Job_df['job_summary']))

In [48]:
# checking again non-ASCII characters 
row_with_id_915 = Job_df.filter(Job_df['ID'] == 915).collect()
row_with_id_915

### Remove extra space

In [49]:
from pyspark.sql.functions import regexp_replace, col, trim

def remove_extra_spaces(df, column_name):
    df = df.withColumn(column_name, regexp_replace(col(column_name), "\\s+", " "))  
    return df.withColumn(column_name, trim(col(column_name))) 

In [50]:
Job_df = remove_extra_spaces(Job_df, "job_summary")

In [ ]:
Job_df.show(5)

### Tokenization

In [ ]:
tokenizer = Tokenizer().setInputCol("job_summary").setOutputCol("Tokens")
Job_df = tokenizer.transform(Job_df)
Job_df.show(5)

### Removing Stopwords

In [ ]:
# removing stopwords using default list
remove_stopwords = StopWordsRemover()
stopwords = remove_stopwords.getStopWords()
print(stopwords[:10])
print(len(stopwords))

remove_stopwords.setInputCol("Tokens").setOutputCol("Tokens stopwords removed")
Job_df = remove_stopwords.transform(Job_df)

In [54]:
# # counting the number of tokens with stopwords removed
Job_df = Job_df.withColumn("Number of tokens", size(Job_df['Tokens']))
Job_df = Job_df.withColumn("Number of tokens After stopwords removed", size(Job_df['Tokens stopwords removed']))
# counting the number of tokens with stopwords removed
Job_df = Job_df.withColumn("Number of stopwords removed", size(Job_df['Tokens stopwords removed']) - size(Job_df['Tokens']) )

In [ ]:
Job_df.show(10)

###  Join the words

To join the words back together after tokenization and stopword removal, you can use the concat_ws function provided by PySpark. Here’s how you can do it:

In [56]:
from pyspark.sql.functions import concat_ws

Job_df = Job_df.withColumn("Cleaned_text", concat_ws(" ", col("Tokens stopwords removed")))

In [ ]:
Job_df.select("Cleaned_text").show(truncate=False)

## Final dataset

In [58]:
Job_df_proces = Job_df.select('Id', "Cleaned_text")

In [ ]:
Job_df_proces.show()

### Creating Shingles

In [61]:
def shingle(text, k):
    shingles = set()
    words = text.split()
    for i in range(len(words) - k + 1):
        shingles.add(' '.join(words[i:i+k]))
    return list(shingles)

In [62]:
k = 2  # Shingle length
shingle_udf = udf(lambda text: shingle(text, k), ArrayType(StringType()))
Job_df_proces = Job_df_proces.withColumn("shingles", shingle_udf(col("Cleaned_text")))

In [ ]:
Job_df_proces.select("shingles").show(5)

#### convert shingles to sparse vectors

In [ ]:
# Flatten the shingles column to get all unique shingles
unique_shingles = Job_df_proces.select(explode("shingles").alias("shingle")).distinct().collect()
shingle_index = {row["shingle"]: idx for idx, row in enumerate(unique_shingles)}

print("Unique shingles and their indices:")
print(shingle_index)

def shingles_to_sparse_vector(shingles):
    indices = sorted([shingle_index[sh] for sh in shingles if sh in shingle_index])
    values = [1.0] * len(indices)
    return Vectors.sparse(len(unique_shingles), indices, values)

# UDF to convert shingles to sparse vectors
sparse_vector_udf = udf(lambda shingles: shingles_to_sparse_vector(shingles), VectorUDT())

Job_df_proces = Job_df_proces.withColumn("features", sparse_vector_udf(col("shingles")))

In [ ]:
# Show the DataFrame with sparse vectors
Job_df_proces.select("features").show(truncate=False)

In [67]:
df = Job_df_proces
# df.show(truncate=False)

## Implementing MinHashLSH

In [ ]:
start = time.time()

# Initialize MinHashLSH
mh = MinHashLSH(inputCol="features", outputCol="hashes", seed=12345, numHashTables=20)
model = mh.fit(df)
print("The hashed dataset where hashed values are stored in the column 'hashes':")
hash = model.transform(df)

# Compute the locality sensitive hashes for the input rows, then perform approximate
# similarity join to Calculate Jaccard Distances.
result = model.approxSimilarityJoin(hash, hash, 0.6, distCol="JaccardDistance").select(
    col("datasetA.id").alias("idA"),
    col("datasetB.id").alias("idB"),
    col("JaccardDistance")
)

# Filter out self-pairs and display the results
result_filtered = result.filter("idA < idB")

end = time.time()
computation_time = round(end - start, 3)
print("Computation time: {} seconds".format(computation_time))

print('------------------------------------')

In [ ]:
type(result_filtered)

In [ ]:
result_filtered.show()

In [ ]:
# showing id pairs with distance < 0.6 sorted in ascending order
result_filtered.sort(result_filtered.JaccardDistance.asc()).show(10)

In [ ]:
result_filtered.sort(result_filtered.JaccardDistance.desc()).show(10)

#### save the result

In [ ]:
# save the result to a file
start = time.time()

size = 5000
result_path = f"/content/gdrive/My Drive/Massive_Data_Project/Result/results_{size}.csv"
result_filtered.write.csv(result_path, header=True)
print('------------- Result Saved ---------------')

end = time.time()
computation_time = round(end - start, 3)
print("Computation time: {} seconds".format(computation_time))
print('------------------------------------')

#### Load the result

In [90]:
# Load the result back from the saved CSV file
size = 5000
loaded_result_path = f"/content/gdrive/My Drive/Massive_Data_Project/Result/results_{size}.csv"
loaded_result = spark.read.csv(loaded_result_path, header=True, inferSchema=True)

In [ ]:
loaded_result.sort(loaded_result.JaccardDistance.asc()).show(10)

In [ ]:
# Filter the results to show only rows with JaccardDistance between 0.2 and 0.3
filtered_result = loaded_result.filter((loaded_result.JaccardDistance >= 0.2) & (loaded_result.JaccardDistance <= 0.3))
filtered_result.sort(filtered_result.JaccardDistance.asc()).show(10)

#### Result of minhash function (hash values)

In [ ]:
hash.show()

#### Sparce vector for first document , id = 160

In [ ]:
hash.first()['features']

#### Signature vector for first document, id = 160
Values inside DenseVector shows value of each hash function.

In [ ]:
# hash value of first Document
hash.first()['hashes']

### Creating New dataframe in order to compare pair document with each other

In [ ]:
df_compare = Job_df.select('Id', "Tokens stopwords removed")
df_compare.show(5)

In [73]:
from pyspark.sql import DataFrame

# Filter the dataset to get the row with the given ID number
def analyze_text_by_id(df: DataFrame, id_number: int):
    row_with_id = df.filter(df['ID'] == id_number).collect()
    print(row_with_id)

    if not row_with_id:
        print(f"No row found with ID {id_number}")
        return

    txt = row_with_id[0][1:][0]

    print(f"Type of txt: {type(txt)}")
    print(f"Length of txt: {len(txt)}")
    print(f"Fourth character in txt: {txt[3]}")

    return txt

In [74]:
# Function to calculate the number of words, common words, and percentage of common words
def analyze_lists(tokens1, tokens2):
    tokens1 = [word for word in tokens1 if word]
    tokens2 = [word for word in tokens2 if word]

    num_words_list1 = len(tokens1)
    num_words_list2 = len(tokens2)
    num_unique_words_list1 = len(set(tokens1))
    num_unique_words_list2 = len(set(tokens2))

    common_words = set(tokens1).intersection(tokens2)
    num_common_words = len(common_words)

    percentage_common_list1 = (num_common_words / num_unique_words_list1) * 100 if num_unique_words_list1 > 0 else 0
    percentage_common_list2 = (num_common_words / num_unique_words_list2) * 100 if num_unique_words_list2 > 0 else 0

    return (num_words_list1, num_words_list2, num_unique_words_list1,
            num_unique_words_list2, num_common_words,
            percentage_common_list1, percentage_common_list2)

### Comparing the Documents with 'ID' number of 1909 & 3014, which have Jaccard distance equal to 0.20



In [ ]:
# Filter the dataset to get the row with ID number 1909
txt1 = analyze_text_by_id(df_compare, 1909)

In [ ]:
# Filter the dataset to get the row with ID number 3014
txt2 = analyze_text_by_id(df_compare, 3014)

In [ ]:
# Comparing number of common words inside Documents with 'ID' number of 1909 & 3014

(num_words_list1, num_words_list2, num_unique_words_list1,
 num_unique_words_list2, num_common_words,
 percentage_common_list1, percentage_common_list2) = analyze_lists(txt1, txt2)

print(f"Number of words in Text_1: {num_words_list1}")
print(f"Number of words in Text_2: {num_words_list2}")
print(f"Number of Unique words in Text_1: {num_unique_words_list1}")
print(f"Number of Unique words in Text_2: {num_unique_words_list2}")
print(f"Number of common Uniqe words: {num_common_words}")
print(f"Percentage of common words in Text_1: {percentage_common_list1:.2f}%")
print(f"Percentage of common words in Text_2: {percentage_common_list2:.2f}%")

### Comparing the Documents with 'ID' number of 3284 & 4955, which have Jaccard distance equal to 0.59

In [ ]:
# Filter the dataset to get the row with ID number 3284
txt5 = analyze_text_by_id(df_compare, 3284)

In [ ]:
# Filter the dataset to get the row with ID number 4955
txt6 = analyze_text_by_id(df_compare, 4955)

In [ ]:
# Comparing number of common words inside Documents with 'ID' number of 3284 & 4955

(num_words_list1, num_words_list2, num_unique_words_list1,
 num_unique_words_list2, num_common_words,
 percentage_common_list1, percentage_common_list2) = analyze_lists(txt5, txt6)

print(f"Number of words in Text_1: {num_words_list1}")
print(f"Number of words in Text_2: {num_words_list2}")
print(f"Number of Unique words in Text_1: {num_unique_words_list1}")
print(f"Number of Unique words in Text_2: {num_unique_words_list2}")
print(f"Number of common Uniqe words: {num_common_words}")
print(f"Percentage of common words in Text_1: {percentage_common_list1:.2f}%")
print(f"Percentage of common words in Text_2: {percentage_common_list2:.2f}%")

### Comparing the Documents with 'ID' number of 503 & 948, which have Jaccard distance equal to 0.



In [ ]:
# Filter the dataset to get the row with ID number 503
txt3 = analyze_text_by_id(df_compare, 503)

In [ ]:
# Filter the dataset to get the row with ID number 948
txt4 = analyze_text_by_id(df_compare, 948)

In [ ]:
# Comparing number of common words inside Documents with 'ID' number of 503 & 948

(num_words_list1, num_words_list2, num_unique_words_list1,
 num_unique_words_list2, num_common_words,
 percentage_common_list1, percentage_common_list2) = analyze_lists(txt3, txt4)

print(f"Number of words in Text_1: {num_words_list1}")
print(f"Number of words in Text_2: {num_words_list2}")
print(f"Number of Unique words in Text_1: {num_unique_words_list1}")
print(f"Number of Unique words in Text_2: {num_unique_words_list2}")
print(f"Number of common Uniqe words: {num_common_words}")
print(f"Percentage of common words in Text_1: {percentage_common_list1:.2f}%")
print(f"Percentage of common words in Text_2: {percentage_common_list2:.2f}%")

## Cheking the Equality of documents:
#### Cheking the Equality of documents with ['ID'] number 503 & 948, which they have Jaccard Distance equal to Zero.

In [ ]:
# ID number 503
row_with_id_503 = Job_Dataset.filter(Job_Dataset['ID'] == 503).collect()
txt = row_with_id_503[0][1:][0]
txt

In [ ]:
# ID number 948
row_with_id_948 = Job_Dataset.filter(Job_Dataset['ID'] == 948).collect()
txt2 = row_with_id_948[0][1:][0]
txt2

In [ ]:
if txt == txt2:
    print('Equal')
else:
    print('Not Equal')